# Data Exploration

**Tian Lou** \
Ohio Education Research Center \
The Ohio State University

**Xiangyu Ren** \
New York University

**Anna-Carolina Haensch** \
University of Maryland \
LMU Munich

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.10257347.svg)](https://doi.org/10.5281/zenodo.10257347)

**This notebook is developed for the [Data Literacy and Evidence Building Executive Class](https://www.socialdatascience.umd.edu/data-literacy).**

**The "Syntucky" data, which is synthetic in nature, is exclusively designed for training exercises. It is not intended to derive meaningful insights or make determinations about real-world populations.**

## Goals

Throughout this series of notebooks, **our overarching goal is to examine job quality of the 2015 Syntucky cohort**. We will use this example project to show you how to 1) explore and understand data structure; 2) link datasets; 3) construct a sample and outcome measurements; 4) create simple but informative visualizations; and 5) use machine learning models.

**The specific research questions we will answer in this notebook are:**
1. How many students are in the 2015 Syntucky cohort?
2. How many students in the 2015 Syntucky cohort graduated with a computer science bachelor's degree?
3. What are the 2015 cohort's mean and median earnings in year 7 (i.e., the seventh year since college entrance)?
4. What are the mean and median earnings in year 7 for the 2015 cohort who graduated with a computer science bachelor's degree?

**After completing this notebook, you should:**
1. Become familiar with commonly used functions in Python Libraries: Pandas and Numpy
2. Be able to explore Syntucky cohort data, including data strucutre and key information
3. Learn how to generate descriptive statistics of the full sample and by subgroups
4. Understand how to check missing values

## 1. Import Data
The first step in every notebook is to use `import` to load libraries in Python. In this notebook, we use two of the most popular Python libraries for data analysis: Pandas and NumPy. They will make our analysis journey smoothe and efficient.

**Pandas**: Pandas specializes in making data manipulation and analysis straightforward and accessible. It allows you to read, organize, and filter data from various file formats easily, using a table-like format called *DataFrame*.

**NumPy**: NumPy is a bit like a high-powered calculator for Python. It's another library that enables efficient numerical computations on large data arrays. When you need to perform mathematical operations on a lot of numbers simultaneously, NumPy is your go-to tool.

In [ ]:
#Load libraries
import pandas as pd
import numpy as np

Before importing the data, let's save the data folder directory in a string variable. This way, it is easy for you to make changes to and keep tracking data direcory.

> Before running the code below, please change <font color='red'> **YOUR DATA DIRECTORY**</font> to your own file path.

In [ ]:
#Define data folder directory
data_directory = 'YOUR DATA DIRECTORY'

Now we can import the 2015 cohort data. We use `pd.read_csv()` to read a CSV (Comma Separated Values) file and convert it into a DataFrame. In the first line of the code, we read in the 2015 Syntucky cohort data and save it into the DataFrame `df_2015`. Now you can manipulate and analyze your data using various Pandas functions on the `df_2015` DataFrame. 

Note that we use `+` to concatenate the data directory defined in the previous step with data file name (also need to be in string format). Then we use `df_2015.head()` to check how the data looks like. The `head()` function returns the first five rows of a DataFrame. In `df_2015`, each row corresponds to a record (or an observation), and each column represents a variable (or a feature). 

In [ ]:
#Read in 2015 Syntucky cohort data 
df_2015 = pd.read_csv(data_directory + 'syntucky_cohort_2015.csv')

#Check the data
df_2015.head()

#### **Checkpoint 1: Import Cohort Data**

Please load the 2013 cohort data and save it in `df_2013`. The csv file of the 2013 cohort data is *"syntucky_cohort_2013.csv"*.

## 2. Understand Cohort Data Structure

In this section, let's take a closer look at the structure of the cohort data. We strongly recommend you to read the codebook first and/or have the codebook opened when going through this section. 
> The codebook is: **constructed-data-codebook.xlsx**

First, **cohort year means the academic year a student first entered college to pursue either an associate's or a bachelor's degree**. For example, the data in "syntucky_cohort_2015.csv" contains students who pursued associate's or bachelor's degrees for the first time in academic year 2015.

Second, **each row in the cohort data represents a person**. We usually refer to this type of data as wide format data. You can find a person's information in one row, including demographics, first enrollment and completion information, as well as enrollment and labor market outcomes from the fifth year to the ninth year since college entrance. 

*One common concern with administrative data is duplicates. A simple way to check duplicates is to check if the number of rows is the same as the number of unique person identifiers in your data.* In this case, we can use `df_2015.shape` to get the number of rows and the number of columns in `df_2015`. Then we can use `df_2015['id'].nunique()` to check the number of unique IDs in `df_2015`. As you can see, the numbers are the same, which means we do not need to worry about duplicates now. :)

In [ ]:
#Check number of rows and number of columns in df_2015
df_2015.shape

In [ ]:
#Check number of unique individuals in df_2015
df_2015['id'].nunique()

Finally, we also want to **check if the data has all the key information we need for our analysis**. For example, since our overarching goal is to examine job quality, we want to ensure that our data contains enough labor market information. In the previous section, we use `df_2015.head()` to browse the data. However, if you scroll the DataFrame to the right, you will notice that not all the columns show in the results. To see what columns a DataFrame has, you can use `df_2015.columns`, it returns the names of the columns in the DataFrame. We can see that the labor market outcomes that are available in our data are earnings (such as `year7_earnings`), number of quarters being employed (such as `year7_ct_qtrs_employed`), number of employers (such as `year7_ct_employers`), etc.

In [ ]:
#Check columns
df_2015.columns

#### **Checkpoint 2: Check Data Structure**

Please check if the data you load in Checkpoint 1 (`df_2013`) has duplicates and if it has the key information you need for your planned analysis.

## 3. Create Subset DataFrames

The Syntucky cohort data include various information that allows us to conduct subgroup analysis. In this example project, we are especially interested in outcomes by major. According to the codebook, there are three columns about a person's major:
> **first_enroll**: The first enrollment major. **first_completion**: The first completion major. **high_completion**: Highest degree major.

Since we are interested in degree major, we can use `.unique()` to check highest degree major. By using the code below, we can see that the possible values for *high_completion* are 'other', 'nursing', 'business', 'education', 'computerscience', or NaN.

In [ ]:
#Check unique values in "high_completion"
df_2015['high_completion'].unique()

Moreover, because one of our research questions is to examine earnings of computer science bachelor's degree earners, we also need **high_completion_label**, the highest degree a person earned within six years after college entrance. In the code below, we use `.groupby()` function to check the number of students by highest degree level and highest degree major. This is a useful method to check sample size before you decide to focus on a specific subgroup population.

In [ ]:
# Check number of students by highest degree level and highest degree major
df_2015.groupby(['high_completion_label', 'high_completion'])['id'].agg(['count']).reset_index()

Next, let's save students who earned a bachelor's degree in computer science in a new DataFrame `df_2015_cs_ba`. This is where the `==` operator comes in. Using `==`, you can create a condition that matches specific values in our dataset, like 'computerscience' in the `high_completion` variable and 'Bachelor' in the `high_completion_label` variable.

`(df_2015['high_completion'] == 'computerscience') & (df_2015['high_completion_label'] == 'Bachelor')` creates a "mask" where each row is either True (if the student's `high_completion` was 'computerscience' and `high_completion_label` is 'Bachelor') or False. Then, adding it to `df_2015[...]` applies this mask to the DataFrame and selects only the rows where this condition is True and write it into `df_2015_cs_ba`. 

> This step is referred to as "subsetting", which is one of the most powerful features in Pandas library. It allows us to "zoom in" on a specific part of our dataset. By creating subsets of our data, you can perform detailed analyses on these specific groups without getting lost in the whole dataset.

We can then use the `.shape` function again to double check if we get the correct sample size.

In [ ]:
#Save students who earned a bachelor's degree in computer science in df_2015_cs_ba
df_2015_cs_ba = df_2015[(df_2015['high_completion'] == 'computerscience') & (df_2015['high_completion_label'] == 'Bachelor')]

#Double check we get the current sample
df_2015_cs_ba.shape

#### **Checkpoint 3: Create a subset of your data**

Please use the subsetting technique you learned in this section to save students in the 2013 cohort who earned a bachelor's degree in education to a new DataFrame `df_2013_ed_ba`.

Are you interested in other degree levels and majors? Try to save them in other DataFrames.

## 4. Generate Descriptive Statistics

After successfully creating subsets of your data, the next step is to delve deeper into these subsets and uncover their secrets. This is where descriptive statistics come in. The `.describe()` function in Pandas can generate various descriptive statistics. For example, the code `df_2015.describe()` provides the following information:

- *count*: the total number of non-null entries in each column.
- *mean*: the average of all entries in each column.
- *std*: the standard deviation of each column, which is a measure of the spread of values.
- *min*: the smallest value in each column.
- *25%* (also known as the first quartile or Q1): the value below which 25% of the data fall.
- *50%* (also known as the second quartile, median or Q2): the middle value of the dataset.
- *75%* (also known as the third quartile or Q3): the value below which 75% of the data fall.
- *max*: the largest value in each column.

In the code below, we specify the columns we want to use to generate summary statistics: **year 5 to year 7 inflation adjusted earnings**. We can see that *count* becomes smaller over time, indicating the number of students with non-null earnings reduces over time. However, `mean` and `50%` (median) increase over time. 
> One possible explanation for these findings is that, over time, as more students graduated from college in Syntucky, they may move to other states and we cannot observe their earnings in other states. However, because they obtained higher levels of degrees, if they decide to stay in Syntucky, they are more likely to find higher paying jobs.

In [ ]:
#Summary Statistics of the 2015 cohort's year 5 to year 7 earnings
df_2015[['year5_earnings', 'year6_earnings', 'year7_earnings']].describe()

We can repeat the same process by using `df_2015_cs_ba`. We can see that the overall trend of computer science bachelor's degree earners is similar to the whole 2015 cohort: fewer non-empty earnings observation over time but increasing mean and median earnings. However, computer science bachelor's degree earners in general have higher mean and median earnings than the whole 2015 cohort.

In [ ]:
#Summary Statistics of the 2015 cohort's year 5 to year 7 earnings
df_2015_cs_ba[['year5_earnings', 'year6_earnings', 'year7_earnings']].describe()

An alternative way to generate descriptive statistics is to use functions in Numpy, such as `np.mean()`, `np.median()` (including null values), `np.nanmedian()` (excluding null values), `np.std()`, `np.var()`, `np.min()`, `np.max()`, and `np.percentile()`. This way, instead of getting all the statistics, you can select which ones you want to see. In the example below, we only look at year 7 average earnings.

In [ ]:
#Use Numpy function np.mean to get
#year 7 median earnings of students who earned bachelor's degree in computer science
np.mean(df_2015_cs_ba['year7_earnings'])

## 5. Check Missing Values

After examining descriptive statistics, it's time to sharpen our focus and look at another vital aspect of our data: the presence of missing values. The transition from descriptive statistics to identifying missing data is like moving from examining the entire forest to checking the health of individual trees. Missing values can distort your analysis and lead to incorrect conclusions. It's crucial to ensure the dataset is complete, robust for analysis, and this is where `isna().sum()` steps into the spotlight.

The combination of `.isna().sum()` is a useful technique in Pandas for identifying the number of missing or `NaN` (Not a Number) values in your DataFrame. 

- `.isna()` is a method that returns a DataFrame of the same shape as your original data, but with True at the positions where the original data has NaN or missing values and False elsewhere.
- `.sum()` then adds up these True values (treated as 1) column by column. The result is a series that shows the count of missing or `NaN` values.

This is an essential part of data cleaning and pre-processing that can significantly impact your analysis and results. Let's try it out with year 7 earnings in `df_2015_cs_ba`.
> Note that **missing values could be in various formats**. Since we know that the Syntucky data has been pre-processed, missing values are mostly in NaN format. However, missing values could also be "0", ".", or other format.

In [ ]:
#Check missing values
df_2015_cs_ba['year7_earnings'].isna().sum()

What percentage of computer science bachelor's degree earners do not have year 7 earnings in our data? In this case, we can use the `print()` function to print several outputs.
> In a Jupyter Notebook, if you have a cell with several lines of code that each produce an output, only the result of the last line is automatically displayed when you run the cell. 

In [ ]:
#Number of 2015 cohort students who earned computer science bachelor's degree
count_cs_ba = df_2015_cs_ba['id'].nunique()

#Number of 2015 cohort students who earned computer science bachelor's degree and 
# have missing year 7 earnings
count_cs_ba_no_y7earn = df_2015_cs_ba['year7_earnings'].isna().sum()

#print results
print("In the 2015 cohort,", count_cs_ba, 
      "students' highest degree level is bachelor's degree in computer science.")
print("Of these students,", count_cs_ba_no_y7earn, "have missing earnings in year 7.")
print("In other words,", round(count_cs_ba_no_y7earn/count_cs_ba, 3), 
      "computer science bachelor's degree earners have missing earnings in year 7.")

#### **Checkpoint 4: Generate Descriptive Statistics and Check Missing Values**

1. Use the 2013 cohort DataFrame you saved in Checkpoint 1, `df_2013`, to get 2013 cohort's mean and median earnings from year 5 - year 7.
2. Use the 2013 education bachelor's degree earners DataFrame you saved in Checkpoint 3, `df_2013_ed_ba`, to get their mean and median earnings from year 5 - year 7.
3. How many students in `df_2013_ed_ba` have missing earnings in year 7? 